In [ ]:
## new train data
import pandas as pd

df_train_full = pd.read_csv("../input/m5-forecasting-accuracy/sales_train_evaluation.csv")
df_train_full.iloc[:, -31:].head()

In [ ]:
## importing packages
import numpy as np
import pandas as pd
from typing import Union
from tqdm.notebook import tqdm

In [ ]:
train_df = df_train_full.iloc[:, :-28].copy()
valid_df = df_train_full.iloc[:, -28:].copy()
train_y = train_df.loc[:, train_df.columns.str.startswith('d_')]
train_y.head()

In [ ]:
train_df.head()

In [ ]:
valid_df.head()#6/1までは1913までのデータしかなかったが、新たに1941まで開示された。

In [ ]:
train_target_columns = train_y.columns.tolist()
print(train_target_columns[-20:])#1900日分のカラム

In [ ]:
train_df['all_id'] = 0
train_df.head()

In [ ]:
id_columns = train_df.loc[:, ~train_df.columns.str.startswith('d_')].columns.tolist()
print(id_columns)#id, item, dept, category, store, state, all

In [ ]:
valid_target_columns = valid_df.loc[:, valid_df.columns.str.startswith('d_')].columns.tolist()
print(valid_target_columns)#検証期間28日分のカラム

In [ ]:
weight_columns = train_y.iloc[:, -28:].columns.tolist()#重みの導出に用いるカラム
print(weight_columns)

In [ ]:
valid_df.head()

In [ ]:
if not all([c in valid_df.columns for c in id_columns]):
    valid_df = pd.concat([train_df[id_columns], valid_df], axis=1, sort=False)
valid_df.head()#valid_dfだけだとd_\d{4}しかなくて寂しいから、_idを加えてあげる。
#6/1に公開されたデータを用いている。
#このvalid_dfの販売数を用いて評価指標の重みを導出する。

In [ ]:
group_ids = (
        'all_id',
        'cat_id',
        'state_id',
        'dept_id',
        'store_id',
        'item_id',
        ['state_id', 'cat_id'],
        ['state_id', 'dept_id'],
        ['store_id', 'cat_id'],
        ['store_id', 'dept_id'],
        ['item_id', 'state_id'],
        ['item_id', 'store_id']
    )

for i, group_id in enumerate(tqdm(group_ids)):
    train_y = train_df.groupby(group_id)[train_target_columns].sum()#検証期間28日分のカラム
    scale = []
    counter = 0
    for _, row in train_y.iterrows():#1900日分のカラム
        series = row.values[np.argmax(row.values != 0):]#sumの値が0ではない日付以降
        scale.append(((series[1:] - series[:-1]) ** 2).mean())#(ある日付-前日)**2

In [ ]:
df_calendar = pd.read_csv("../input/m5-forecasting-accuracy/calendar.csv")
df_prices = pd.read_csv("../input/m5-forecasting-accuracy/sell_prices.csv")
day_to_week = df_calendar.set_index('d')['wm_yr_wk']
day_to_week.head()

In [ ]:
weight_df = df_train_full[['item_id', 'store_id'] + weight_columns].set_index(['item_id', 'store_id'])
weight_df.head()#item_id, store_idをindexにした

In [ ]:
#横持ちのマトリックスを縦持ちに変更する。
weight_df = weight_df.stack().reset_index().rename(columns={'level_2': 'd', 0: 'value'})
weight_df.head(20)#valueは販売数

In [ ]:
weight_df['wm_yr_wk'] = weight_df['d'].map(day_to_week)
weight_df.head()

In [ ]:
weight_df = weight_df.merge(df_prices, how='left', on=['item_id', 'store_id', 'wm_yr_wk'])
weight_df.head()

In [ ]:
weight_df['value'] = weight_df['value'] * weight_df['sell_price']
weight_df.head()
#valueは販売数*販売価格

In [ ]:
#今度は逆に縦持ちから横持ちへと変換。
weight_df = weight_df.set_index(['item_id', 'store_id', 'd']).unstack(level=2)['value']
weight_df.head()
#valueは購入総額
#unstack(level=2)とすると、indexの左から二つまでが新たなindexとなり、右の一つがvalueとなる。

In [ ]:
weight_df = weight_df.loc[zip(train_df.item_id, train_df.store_id), :].reset_index(drop=True)
weight_df.head()
#valueだけ取り出す。

In [ ]:
weight_df = pd.concat([train_df[id_columns], weight_df], axis=1, sort=False)
weight_df.head()

In [ ]:
lv_weight = weight_df.groupby(group_id)[weight_columns].sum().sum(axis=1)#縦に足してから、横に足す。
lv_weight.head()

In [ ]:
lv_weight.sum()

In [ ]:
## evaluation metric
## from https://www.kaggle.com/c/m5-forecasting-accuracy/discussion/133834 and edited to get scores at all levels
class WRMSSEEvaluator(object):

    def __init__(self, train_df: pd.DataFrame, valid_df: pd.DataFrame, calendar: pd.DataFrame, prices: pd.DataFrame):
        train_y = train_df.loc[:, train_df.columns.str.startswith('d_')]
        train_target_columns = train_y.columns.tolist()
        weight_columns = train_y.iloc[:, -28:].columns.tolist()

        train_df['all_id'] = 0  # for lv1 aggregation

        id_columns = train_df.loc[:, ~train_df.columns.str.startswith('d_')].columns.tolist()
        valid_target_columns = valid_df.loc[:, valid_df.columns.str.startswith('d_')].columns.tolist()#1914~

        if not all([c in valid_df.columns for c in id_columns]):
            valid_df = pd.concat([train_df[id_columns], valid_df], axis=1, sort=False)#1914~

        self.train_df = train_df
        self.valid_df = valid_df
        self.calendar = calendar
        self.prices = prices

        self.weight_columns = weight_columns
        self.id_columns = id_columns
        self.valid_target_columns = valid_target_columns

        weight_df = self.get_weight_df()

        self.group_ids = (
            'all_id',
            'cat_id',
            'state_id',
            'dept_id',
            'store_id',
            'item_id',
            ['state_id', 'cat_id'],
            ['state_id', 'dept_id'],
            ['store_id', 'cat_id'],
            ['store_id', 'dept_id'],
            ['item_id', 'state_id'],
            ['item_id', 'store_id']
        )

        for i, group_id in enumerate(tqdm(self.group_ids)):
            #scaleの導出には1900日全てを使用する。
            train_y = train_df.groupby(group_id)[train_target_columns].sum()#各日にちのvalueを縦方向に足し合わせる
            scale = []
            for _, row in train_y.iterrows():
                series = row.values[np.argmax(row.values != 0):]#例えば3個のstateの各日にちごとの売り上げ数の総和
                scale.append(((series[1:] - series[:-1]) ** 2).mean())#前日との差の二乗平均がscaleになる
            setattr(self, f'lv{i + 1}_scale', np.array(scale))#前日とのmean_sequre_error, groupbyの項目数だけ存在する。
            setattr(self, f'lv{i + 1}_train_df', train_y)
            setattr(self, f'lv{i + 1}_valid_df', valid_df.groupby(group_id)[valid_target_columns].sum())#28日分のid

            lv_weight = weight_df.groupby(group_id)[weight_columns].sum().sum(axis=1)#縦に足してから、横に足している
            setattr(self, f'lv{i + 1}_weight', lv_weight / lv_weight.sum())#lv_weightを正規化する。#縦に足してから、横に足して、それをさらに足して総和

    def get_weight_df(self) -> pd.DataFrame:
        day_to_week = self.calendar.set_index('d')['wm_yr_wk'].to_dict()
        weight_df = self.train_df[['item_id', 'store_id'] + self.weight_columns].set_index(['item_id', 'store_id'])
        weight_df = weight_df.stack().reset_index().rename(columns={'level_2': 'd', 0: 'value'})#item, store, dayで売り上げ個数を集計
        weight_df['wm_yr_wk'] = weight_df['d'].map(day_to_week)#週カラムを新たに追加(集計はしていない)

        weight_df = weight_df.merge(self.prices, how='left', on=['item_id', 'store_id', 'wm_yr_wk'])#item, store, weekごとのprice
        weight_df['value'] = weight_df['value'] * weight_df['sell_price']#valueはitem, store, dayごとの各商品の販売総額
        weight_df = weight_df.set_index(['item_id', 'store_id', 'd']).unstack(level=2)['value']#横持ちに戻す。valueは商品総額
        weight_df = weight_df.loc[zip(self.train_df.item_id, self.train_df.store_id), :].reset_index(drop=True)#indexを削除する
        weight_df = pd.concat([self.train_df[self.id_columns], weight_df], axis=1, sort=False)#元のidカラムたちと結合して
        return weight_df#各日にちの販売総額がvalueとして格納されている。

    def rmsse(self, valid_preds: pd.DataFrame, lv: int) -> pd.Series:
        valid_y = getattr(self, f'lv{lv}_valid_df')#ここで1941のカラムを使用しているから、validの値は計算に必要である。
        score = ((valid_y - valid_preds) ** 2).mean(axis=1)#二乗誤差平均#groupbyで集計されたものから,valid_preds(しっかりとgroupbyされている)を引く
        scale = getattr(self, f'lv{lv}_scale')
        return (score / scale).map(np.sqrt)

    def score(self, valid_preds: Union[pd.DataFrame, np.ndarray]):
        assert self.valid_df[self.valid_target_columns].shape == valid_preds.shape

        if isinstance(valid_preds, np.ndarray):
            valid_preds = pd.DataFrame(valid_preds, columns=self.valid_target_columns)

        valid_preds = pd.concat([self.valid_df[self.id_columns], valid_preds], axis=1, sort=False)

        group_ids = []
        all_scores = []
        for i, group_id in enumerate(self.group_ids):
            #rmsseの計算にはweightはいらないが、validの値が必要である。
            lv_scores = self.rmsse(valid_preds.groupby(group_id)[self.valid_target_columns].sum(), i + 1)#あらかじめvalid_predsをgroupbyしてくれている。
            weight = getattr(self, f'lv{i + 1}_weight')
            lv_scores = pd.concat([weight, lv_scores], axis=1, sort=False).prod(axis=1)#prodでさりげなく重みを掛け合わせている。
            group_ids.append(group_id)
            all_scores.append(lv_scores.sum())

        return group_ids, all_scores


In [ ]:
## public LB rank
def get_lb_rank(score):
    df_lb = pd.read_csv("../input/m5-accuracy-final-public-lb/m5-forecasting-accuracy-publicleaderboard-rank.csv")
    return (df_lb.Score <= score).sum() + 1

In [ ]:
## reading data
df_calendar = pd.read_csv("../input/m5-forecasting-accuracy/calendar.csv")
df_prices = pd.read_csv("../input/m5-forecasting-accuracy/sell_prices.csv")
df_sample_submission = pd.read_csv("../input/m5-forecasting-accuracy/sample_submission.csv")
df_sample_submission["order"] = range(df_sample_submission.shape[0])

df_train = df_train_full.iloc[:, :-28]
df_valid = df_train_full.iloc[:, -28:]
evaluator = WRMSSEEvaluator(df_train, df_valid, df_calendar, df_prices)


In [ ]:
evaluator.lv5_scale

In [ ]:
## structure of validation data
preds_valid = df_valid.copy() + np.random.randint(100, size = df_valid.shape)
preds_valid.head()


In [ ]:
## evaluating random submission
groups, scores = evaluator.score(preds_valid)

score_public_lb = np.mean(scores)
score_public_rank = get_lb_rank(score_public_lb)

for i in range(len(groups)):
    print(f"Score for group {groups[i]}: {round(scores[i], 5)}")

print(f"\nPublic LB Score: {round(score_public_lb, 5)}")
print(f"Public LB Rank: {score_public_rank}")

In [ ]:
preds_valid = pd.read_csv("../input/m5-three-shades-of-dark-darker-magic/submission_v1.csv")
preds_valid = preds_valid[preds_valid.id.str.contains("validation")]
preds_valid.head()

In [ ]:
df_sample_submission.head()

In [ ]:
preds_valid = preds_valid.merge(df_sample_submission[["id", "order"]], on = "id").sort_values("order")#.drop(["id", "order"], axis = 1).reset_index(drop = True)
#df_sample_submissionはid, order列のみしか使用していない。
preds_valid.head()
preds_valid.shape

In [ ]:
#submissionファイルを読み込んで、validationという文字をidに含んでいる行のみを抽出して、カラムをrename
preds_valid = preds_valid.merge(df_sample_submission[["id", "order"]], on = "id").sort_values("order").drop(["id", "order"], axis = 1).reset_index(drop = True)
preds_valid.rename(columns = {
    "F1": "d_1914", "F2": "d_1915", "F3": "d_1916", "F4": "d_1917", "F5": "d_1918", "F6": "d_1919", "F7": "d_1920",
    "F8": "d_1921", "F9": "d_1922", "F10": "d_1923", "F11": "d_1924", "F12": "d_1925", "F13": "d_1926", "F14": "d_1927",
    "F15": "d_1928", "F16": "d_1929", "F17": "d_1930", "F18": "d_1931", "F19": "d_1932", "F20": "d_1933", "F21": "d_1934",
    "F22": "d_1935", "F23": "d_1936", "F24": "d_1937", "F25": "d_1938", "F26": "d_1939", "F27": "d_1940", "F28": "d_1941"
}, inplace = True)

groups, scores = evaluator.score(preds_valid)

score_public_lb = np.mean(scores)
score_public_rank = get_lb_rank(score_public_lb)

for i in range(len(groups)):
    print(f"Score for group {groups[i]}: {round(scores[i], 5)}")

print(f"\nPublic LB Score: {round(score_public_lb, 5)}")
print(f"Public LB Rank: {score_public_rank}")


In [ ]:
## evaluating submission from public kernel M5 - Witch Time
## from https://www.kaggle.com/kyakovlev/m5-witch-time
preds_valid = pd.read_csv("../input/m5-witch-time/submission.csv")
preds_valid = preds_valid[preds_valid.id.str.contains("validation")]
preds_valid = preds_valid.merge(df_sample_submission[["id", "order"]], on = "id").sort_values("order").drop(["id", "order"], axis = 1).reset_index(drop = True)
preds_valid.rename(columns = {
    "F1": "d_1914", "F2": "d_1915", "F3": "d_1916", "F4": "d_1917", "F5": "d_1918", "F6": "d_1919", "F7": "d_1920",
    "F8": "d_1921", "F9": "d_1922", "F10": "d_1923", "F11": "d_1924", "F12": "d_1925", "F13": "d_1926", "F14": "d_1927",
    "F15": "d_1928", "F16": "d_1929", "F17": "d_1930", "F18": "d_1931", "F19": "d_1932", "F20": "d_1933", "F21": "d_1934",
    "F22": "d_1935", "F23": "d_1936", "F24": "d_1937", "F25": "d_1938", "F26": "d_1939", "F27": "d_1940", "F28": "d_1941"
}, inplace = True)

groups, scores = evaluator.score(preds_valid)#valid_dfを上記のように処理したのち、score関数に渡せば、lbを導出してくれる。

score_public_lb = np.mean(scores)
score_public_rank = get_lb_rank(score_public_lb)

for i in range(len(groups)):
    print(f"Score for group {groups[i]}: {round(scores[i], 5)}")

print(f"\nPublic LB Score: {round(score_public_lb, 5)}")
print(f"Public LB Rank: {score_public_rank}")


In [ ]:
df_train_full = pd.read_csv("../input/m5-forecasting-accuracy/sales_train_evaluation.csv")
df_calendar = pd.read_csv("../input/m5-forecasting-accuracy/calendar.csv")
df_prices = pd.read_csv("../input/m5-forecasting-accuracy/sell_prices.csv")
df_sample_submission = pd.read_csv("../input/m5-forecasting-accuracy/sample_submission.csv")
df_sample_submission["order"] = range(df_sample_submission.shape[0])

df_train = df_train_full.iloc[:, :-28]
df_valid = df_train_full.iloc[:, -28:]
evaluator = WRMSSEEvaluator(df_train, df_valid, df_calendar, df_prices)#df_validは販売数も判明していないと計算できない。

In [ ]:
def get_lb_rank(score):
    df_lb = pd.read_csv("../input/m5-accuracy-final-public-lb/m5-forecasting-accuracy-publicleaderboard-rank.csv")
    return (df_lb.Score <= score).sum() + 1

def calc_lb(preds_path, evaluator):
    preds_valid = pd.read_csv(preds_path)
    preds_valid = preds_valid[preds_valid.id.str.contains("validation")]
    preds_valid = preds_valid.merge(df_sample_submission[["id", "order"]], on = "id").sort_values("order").drop(["id", "order"], axis = 1).reset_index(drop = True)
    preds_valid.rename(columns = {
        "F1": "d_1914", "F2": "d_1915", "F3": "d_1916", "F4": "d_1917", "F5": "d_1918", "F6": "d_1919", "F7": "d_1920",
        "F8": "d_1921", "F9": "d_1922", "F10": "d_1923", "F11": "d_1924", "F12": "d_1925", "F13": "d_1926", "F14": "d_1927",
        "F15": "d_1928", "F16": "d_1929", "F17": "d_1930", "F18": "d_1931", "F19": "d_1932", "F20": "d_1933", "F21": "d_1934",
        "F22": "d_1935", "F23": "d_1936", "F24": "d_1937", "F25": "d_1938", "F26": "d_1939", "F27": "d_1940", "F28": "d_1941"
    }, inplace = True)

    groups, scores = evaluator.score(preds_valid)#valid_dfを上記のように処理したのち、score関数に渡せば、lbを導出してくれる。

    score_public_lb = np.mean(scores)
    score_public_rank = get_lb_rank(score_public_lb)
    
    return score_public_lb, score_public_rank

In [ ]:
#３ヶ月分のlbの平均値が高いモデルを提出する?
#同月前年の重みを使用する?
#上の重みをNNに組み込むためにはどうすればいいのか?
#実測値から重みを計算->valid_predsに適用(実際の28日分の重みが判明しない限りは、scale, weightは計算できない)
#(先月の売り上げからは計算できない。)